## Transforming Zonas de Patrullaje 2018 - clean_ZonasPatrullaje.csv 

This notebook is going to be used to extract and transform the clean dataset by adding new columns that we need for our investigation. Some columns we need are 